In [81]:
from sklearn.datasets import load_breast_cancer
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings

warnings.filterwarnings("ignore", category=FutureWarning)

In [82]:
# Load the dataset and initialize the base predictor
cancer = load_breast_cancer()
d_tree = DecisionTreeClassifier(max_depth = 4, random_state = 0)

In [83]:
# Split dataset into train and test
X_train, X_test, y_train, y_test = train_test_split(cancer.data, cancer.target, stratify = cancer.target, random_state = 42)

# Base Decision Tree Predictor

In [84]:
d_tree.fit(X_train, y_train)
print("Train Set Accuracy = ", d_tree.score(X_train, y_train))
print("Test Set Accuracy = ", d_tree.score(X_test, y_test))

Train Set Accuracy =  0.9882629107981221
Test Set Accuracy =  0.951048951048951


# Perform Bagging using Decision Tree as the base predictor

In [85]:
bagging_classifier = BaggingClassifier(base_estimator = d_tree, n_estimators = 10, random_state = 42)
bagging_classifier.fit(X_train, y_train)
y_pred = bagging_classifier.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Bagging model = ", bagging_accuracy)

Accuracy of the Bagging model =  0.951048951048951


# Experimenting with the parameters

In [88]:
# Experimenting with the no. of estimators
for i in [1, 2, 3, 4, 5, 10, 50, 100]:
    bagging_classifier = BaggingClassifier(base_estimator = d_tree, n_estimators = i, random_state = 42)
    bagging_classifier.fit(X_train, y_train)
    y_pred = bagging_classifier.predict(X_test)
    bagging_accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the Bagging model when the no. of estimators = {i} is {bagging_accuracy}")

Accuracy of the Bagging model when the no. of estimators = 1 is 0.916083916083916
Accuracy of the Bagging model when the no. of estimators = 2 is 0.965034965034965
Accuracy of the Bagging model when the no. of estimators = 3 is 0.965034965034965
Accuracy of the Bagging model when the no. of estimators = 4 is 0.958041958041958
Accuracy of the Bagging model when the no. of estimators = 5 is 0.965034965034965
Accuracy of the Bagging model when the no. of estimators = 10 is 0.951048951048951
Accuracy of the Bagging model when the no. of estimators = 50 is 0.951048951048951
Accuracy of the Bagging model when the no. of estimators = 100 is 0.958041958041958


2

In [108]:
# Let's set no.of estimators to 3 (as it has the best accuracy) and now experiment with the sample size
for i in [1, 5, 10, 50, 100]:
    bagging_classifier = BaggingClassifier(base_estimator = d_tree, n_estimators = 3, max_samples = i, random_state = 42)
    bagging_classifier.fit(X_train, y_train)
    y_pred = bagging_classifier.predict(X_test)
    bagging_accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the Bagging model when the maximum size of sample = {i} is {bagging_accuracy}")

Accuracy of the Bagging model when the maximum size of sample = 1 is 0.6293706293706294
Accuracy of the Bagging model when the maximum size of sample = 5 is 0.8461538461538461
Accuracy of the Bagging model when the maximum size of sample = 10 is 0.8531468531468531
Accuracy of the Bagging model when the maximum size of sample = 50 is 0.9370629370629371
Accuracy of the Bagging model when the maximum size of sample = 100 is 0.965034965034965


In [113]:
# Now let's set maximum size of sample to 100 (as this has the best accuracy) and experiment with samples being drawn with/without replacement
# By default, bootstrap is set to true. So, we are more interested in the results of bootstrap = False
bagging_classifier = BaggingClassifier(base_estimator = d_tree, n_estimators = 3, max_samples = 100, random_state = 42, bootstrap = True)
bagging_classifier.fit(X_train, y_train)
y_pred = bagging_classifier.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Bagging model when the maximum size of sample = {i} is {bagging_accuracy}")

bagging_classifier = BaggingClassifier(base_estimator = d_tree, n_estimators = 3, max_samples = i, random_state = 42, bootstrap = False)
bagging_classifier.fit(X_train, y_train)
y_pred = bagging_classifier.predict(X_test)
bagging_accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy of the Bagging model when the maximum size of sample = {i} is {bagging_accuracy}")

Accuracy of the Bagging model when the maximum size of sample = 100 is 0.965034965034965
Accuracy of the Bagging model when the maximum size of sample = 100 is 0.972027972027972


__Based on the accuracy scores by experimenting with the number of estimators, maximum sample size, and with/without replacement in our samples, we can make a few interesting observations. One is that the accuracy score does not vary based on the number of estimators going incrementally or decrementally. Which means that we can also get a good accuracy score with a fewer estimators also. If we look at the results above, we see that be it 2 estimators or 5 estimators, we are able to yield the same maximum accuracy. But one thing to note was upon further experimentation, I could find that all the number of estimators have an unique impact on the accuracy when sample is also changed.__

__Next with the maximum sample size, we see that there is a trend going on incrementally as we change the sample size incrementally as well. However, we can't conclude this without experimenting more with sample sizes. Upon trying sample size = 110, the accuracy turned out to be lesser than when sample size = 100.__

__Lastly, sampling with or without replacement is an important method of experimentation statisticians use. It's an interesting option to experiment with to see how our results change. Generally, whether replacement should be done or not depends on various factors like population size. For our problem, we can experiment without replacement as well. But, upon setting bootstrap to False, it looks like our accuracy has gone down. Or at least, when we don't change the other parameter values. But if we change the other parameters, we find that the accuracy can also be same or even improve. For example, when I experimented bootstrap = False, but changed the no.of estimators from 3 to 8, the accuracy had gone up from 0.96 to 0.97.__

__What I can say one thing for sure is that there is no patter to accuracy being proportional or inverserly proportional to any of the parameters I experimented. It seems like in practice, it's best to find a suitable value for each parameter by experimenting with them individually but finally also playing around all parameters together, can prove to be worthwhile because there's still a high chance you can continue to improve your accuracy even after individually experimenting with parameters to find their optimal values in your problem.__

# Perform AdaBoost using Decision Tree as the base predictor

In [87]:
adaboost_classifier = AdaBoostClassifier(base_estimator=d_tree, n_estimators=100, random_state=42)
adaboost_classifier.fit(X_train, y_train)
y_pred = adaboost_classifier.predict(X_test)
adaboost_accuracy = accuracy_score(y_test, y_pred)
print("Accuracy of the Adaboost model = ", adaboost_accuracy)

Accuracy of the Adaboost model =  0.951048951048951


# Experimenting with the parameters

In [114]:
# Experimenting with the no. of estimators
for i in [1, 2, 3, 4, 5, 10, 50, 100]:
    adaboost_classifier = AdaBoostClassifier(base_estimator = d_tree, n_estimators = i, random_state=42)
    adaboost_classifier.fit(X_train, y_train)
    y_pred = adaboost_classifier.predict(X_test)
    adaboost_accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the Adaboost model when the no. of estimators = {i} is {adaboost_accuracy}")

Accuracy of the Adaboost model when the no. of estimators = 1 is 0.9440559440559441
Accuracy of the Adaboost model when the no. of estimators = 2 is 0.9020979020979021
Accuracy of the Adaboost model when the no. of estimators = 3 is 0.916083916083916
Accuracy of the Adaboost model when the no. of estimators = 4 is 0.9370629370629371
Accuracy of the Adaboost model when the no. of estimators = 5 is 0.916083916083916
Accuracy of the Adaboost model when the no. of estimators = 10 is 0.9440559440559441
Accuracy of the Adaboost model when the no. of estimators = 50 is 0.965034965034965
Accuracy of the Adaboost model when the no. of estimators = 100 is 0.951048951048951


In [123]:
# Let's set no.of estimators to 50 (as it has the best accuracy) and now experiment with the learning rate
for i in [0.01, 0.1, 0.5, 1.0, 2.0, 5.0]:
    adaboost_classifier = AdaBoostClassifier(base_estimator = d_tree, n_estimators = 50, learning_rate = i, random_state = 42)
    adaboost_classifier.fit(X_train, y_train)
    y_pred = adaboost_classifier.predict(X_test)
    adaboost_accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the Adaboost model when the learning rate = {i} is {adaboost_accuracy}")

Accuracy of the Adaboost model when the learning rate = 0.01 is 0.9300699300699301
Accuracy of the Adaboost model when the learning rate = 0.1 is 0.951048951048951
Accuracy of the Adaboost model when the learning rate = 0.5 is 0.9440559440559441
Accuracy of the Adaboost model when the learning rate = 1.0 is 0.965034965034965
Accuracy of the Adaboost model when the learning rate = 2.0 is 0.965034965034965
Accuracy of the Adaboost model when the learning rate = 5.0 is 0.951048951048951


In [124]:
# It seems to be that a learning a rate of 1 works out really well. Finally, let's experiment with the random state
for i in [1, 2, 3, 5, 10, 42, 50 , 100]:
    adaboost_classifier = AdaBoostClassifier(base_estimator = d_tree, n_estimators = 50, learning_rate = 1, random_state = i)
    adaboost_classifier.fit(X_train, y_train)
    y_pred = adaboost_classifier.predict(X_test)
    adaboost_accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy of the Adaboost model when the random state = {i} is {adaboost_accuracy}")

Accuracy of the Adaboost model when the random state = 1 is 0.958041958041958
Accuracy of the Adaboost model when the random state = 2 is 0.965034965034965
Accuracy of the Adaboost model when the random state = 3 is 0.965034965034965
Accuracy of the Adaboost model when the random state = 5 is 0.9440559440559441
Accuracy of the Adaboost model when the random state = 10 is 0.965034965034965
Accuracy of the Adaboost model when the random state = 42 is 0.965034965034965
Accuracy of the Adaboost model when the random state = 50 is 0.972027972027972
Accuracy of the Adaboost model when the random state = 100 is 0.9300699300699301


__We have some very interesting results to work with here. The first thing to take notice here is that all of them don't seem to have any pattern in accuracy changes as the parameters are modified, which is expected as seen in the boosting decision tree earlier. The next thing we can take notice is that we can yield similar accuracies with different number of estimators. For example, 1 and 10 yield the same accuracy values.__

__Next we observe very interesting results with learning rate. We see that different learning rates yield different results. Generally, when the learning rate is very high, our model tends take more time to find the global minima and sometimes doesn't even find it. But for our problem we see that the accuracy improves with a very high learning rate such as 2 and 2.5 also.__

__Finally, of the remaining parameters in sklearn's adaboost model, one more parameter that is worthwhile experimenting is random_state although it's just how randomizied do you want the samples to be. We see that the highest accuracy is when the random_state is 50. However, it might be better to experiment with other parameters as randomization generally needs more evaluation by understanding the complete working of the model, the problem we are working with, how the randomization works, and etc. For our use case, we may expirement and see how well it generalizes on unseen data to decide if random_state is valuable for identifying a malignant or benign breast cancer tumour.__